In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import torch
import torch.nn.functional as F

In [39]:
# Configuration
torch.manual_seed(0)
batch_size = 1
seq_len = 10
d_model = 768
num_heads = 12
head_dim = d_model // num_heads

# Random input sequence (batch_size, seq_len, d_model)
X = torch.randn(batch_size, seq_len, d_model, requires_grad=True)
# Define MultiheadAttention module
mha = torch.nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, bias=False, batch_first=True)
# Forward pass to get attention weights
attn_output, attn_weights = mha(X, X, X, need_weights=True, average_attn_weights=False)
# attn_weights shape: (batch_size, num_heads, seq_len, seq_len)

In [40]:
print(attn_weights.shape)
print(attn_output.shape)

torch.Size([1, 12, 10, 10])
torch.Size([1, 10, 768])


### Compute the QK and VO circuit full weights

In [41]:
from einops import einsum, rearrange

In [42]:
Wqkv = mha.in_proj_weight.data.detach()
Wq = Wqkv[:d_model, :]
Wk = Wqkv[d_model:2*d_model, :]
Wv = Wqkv[2*d_model:, :]
WO = mha.out_proj.weight.data.detach()
print(Wq.shape, Wk.shape, Wv.shape, WO.shape)

torch.Size([768, 768]) torch.Size([768, 768]) torch.Size([768, 768]) torch.Size([768, 768])


In [43]:
Wq_sep = rearrange(Wq, "(num_head head_dim) hidden_dim -> num_head head_dim hidden_dim", num_head=num_heads, hidden_dim=d_model)
Wk_sep = rearrange(Wk, "(num_head head_dim) hidden_dim -> num_head head_dim hidden_dim", num_head=num_heads, hidden_dim=d_model)
Wqk = einsum(Wq_sep, Wk_sep, "H hd mq, H hd mk -> H mq mk")
Wqk.shape

torch.Size([12, 768, 768])

In [44]:
Wv_sep = rearrange(Wv, "(num_head head_dim) hidden_dim -> num_head head_dim hidden_dim", num_head=num_heads, hidden_dim=d_model)
Wo_sep = rearrange(WO, "hidden_dim (num_head head_dim) -> num_head head_dim hidden_dim", num_head=num_heads, hidden_dim=d_model)
Wov = einsum(Wo_sep, Wv_sep, "H hd do, H hd dv -> H do dv")
Wov.shape

torch.Size([12, 768, 768])

### Compute the Token to Token Jacobian

#### Full Jacobian consider both QK and VO

In [93]:
# Choose token indices and head to inspect
i = 0  # query position
j = 2  # key position
def token_mapping(xj, ):
    """In this version, we consider jacobian through both the QK and VO circuit"""
    X2 = X.clone().detach().requires_grad_(False)
    X2[0, j, :] = xj
    attn_output, attn_weights = mha(X2, X2, X2, need_weights=True, average_attn_weights=False)
    return attn_output[0, i, :]

X = torch.randn(batch_size, seq_len, d_model, requires_grad=True)
with torch.no_grad():
    attn_output, attn_weights = mha(X, X, X, need_weights=True, average_attn_weights=False)

numeric_jac = torch.autograd.functional.jacobian(token_mapping, X[0, j])

First consider the VO contribution

In [94]:
attn_weights[0, :, i, j]

tensor([0.0834, 0.1054, 0.1300, 0.1418, 0.1276, 0.1216, 0.1146, 0.0293, 0.0515,
        0.0589, 0.1084, 0.0608])

In [95]:
ov_weighted = einsum(attn_weights[0, :, i, j], Wov, "H, H do dv -> do dv")
ov_weighted

tensor([[-6.3718e-04,  2.5845e-03,  4.5132e-04,  ..., -2.5611e-03,
         -1.2094e-03, -1.7672e-04],
        [ 2.2032e-03, -1.5285e-04, -5.5944e-04,  ...,  3.3561e-04,
          3.9139e-05,  7.1109e-04],
        [-1.1190e-03, -4.5439e-04, -1.6138e-03,  ..., -9.8907e-04,
         -1.5399e-03, -9.5641e-04],
        ...,
        [ 7.6975e-04,  3.7202e-04,  1.2910e-03,  ...,  1.7296e-04,
         -1.2256e-03, -1.3560e-03],
        [ 1.8892e-03, -2.5997e-03,  2.0197e-04,  ...,  1.5213e-03,
         -1.5377e-03, -7.2043e-04],
        [ 9.7164e-04, -2.9589e-03,  3.5719e-03,  ..., -3.1926e-03,
          4.2033e-05,  3.2716e-04]])

In [96]:
# similar to numeric_jac
torch.corrcoef(torch.stack([numeric_jac.flatten(), ov_weighted.flatten()]))[0, 1]

tensor(0.9025)

In [97]:
# Compute explained variance between numeric Jacobian and OV circuit
x = numeric_jac.flatten()
y = ov_weighted.flatten()

# Calculate correlation coefficient
corr = torch.corrcoef(torch.stack([x, y]))[0, 1]

# Explained variance is the square of the correlation coefficient
explained_variance = corr**2

print(f"Correlation: {corr:.4f}")
print(f"Explained variance: {explained_variance:.4f} ({explained_variance*100:.2f}%)")

# Return the flattened tensors for inspection
x, y

Correlation: 0.9025
Explained variance: 0.8146 (81.46%)


(tensor([-0.0002,  0.0034,  0.0014,  ..., -0.0046, -0.0013, -0.0008]),
 tensor([-6.3718e-04,  2.5845e-03,  4.5132e-04,  ..., -3.1926e-03,
          4.2033e-05,  3.2716e-04]))

Consider 2nd order term

In [ ]:
X_Hqk = einsum(X[0, :, :].detach(), Wqk, "T dq, H dq dk -> H T dk")
X_Hov = einsum(X[0, :, :].detach(), Wov, "T dv, H do dv -> H T do")

In [100]:
qkvo_2ndorder = einsum(attn_weights[0, :, i, j], X_Hqk[:, i, :], X_Hov[:, j, :], "H, H dk, H do -> do dk") / math.sqrt(head_dim)

In [102]:
(ov_weighted + qkvo_2ndorder)

tensor([[-1.6108e-03,  1.8074e-03, -1.0277e-03,  ..., -2.1780e-03,
         -2.0945e-03, -4.3373e-04],
        [ 1.4059e-03,  2.8603e-04, -2.3029e-05,  ...,  2.9454e-04,
          1.8817e-04,  7.8253e-04],
        [-1.1524e-03, -9.2582e-04, -2.6892e-03,  ..., -2.0970e-03,
         -2.0933e-03, -6.3409e-04],
        ...,
        [ 2.0125e-03,  3.7544e-04,  1.9074e-03,  ...,  2.6615e-04,
         -2.9684e-04, -1.4271e-03],
        [ 2.6609e-03, -1.3744e-03,  1.4288e-03,  ...,  2.7168e-03,
         -2.6248e-04, -2.2816e-03],
        [ 5.1004e-04, -2.5448e-03,  3.9063e-03,  ..., -2.8003e-03,
         -5.4014e-05,  7.9646e-04]])

In [103]:
numeric_jac

tensor([[-1.9550e-04,  3.3597e-03,  1.3808e-03,  ..., -1.3318e-03,
         -3.6361e-04,  9.5674e-06],
        [ 1.4560e-03,  9.9350e-04, -7.1403e-05,  ...,  1.6253e-03,
         -1.1831e-05,  8.6948e-04],
        [-1.4688e-03, -6.9230e-04, -1.2832e-03,  ..., -1.1764e-03,
         -1.8379e-03, -1.0612e-03],
        ...,
        [-2.3402e-04,  8.5856e-04,  2.0129e-03,  ...,  1.0299e-03,
         -1.8951e-03, -1.5799e-03],
        [ 2.3125e-03, -2.5183e-03,  1.2634e-04,  ...,  1.7311e-03,
         -1.9201e-03, -1.8885e-04],
        [ 2.2454e-04, -4.0103e-03,  3.2039e-03,  ..., -4.5513e-03,
         -1.2610e-03, -8.1849e-04]])

In [105]:
# Compute explained variance between numeric Jacobian and OV circuit
x = numeric_jac.flatten()
y = (ov_weighted + qkvo_2ndorder).flatten()

# Calculate correlation coefficient
corr = torch.corrcoef(torch.stack([x, y]))[0, 1]
print(f"Correlation: {corr:.4f}")

Correlation: 0.8129


Third term 

In [106]:
qkvo_jl_2ndorder = einsum(attn_weights[0, :, i, j], attn_weights[0, :, i, :], X_Hov[:, :, :], X_Hqk[:, i, :], "H, H Tl, H Tl do, H dk -> do dk") / math.sqrt(head_dim)
qkvo_jl_2ndorder.shape

torch.Size([768, 768])

In [108]:
qkvo_jl_2ndorder

tensor([[-3.9767e-04,  1.1089e-05, -3.3760e-04,  ..., -2.4968e-04,
          5.0448e-05,  1.1504e-04],
        [-1.6807e-04, -5.2521e-04, -3.0439e-04,  ..., -1.9424e-04,
         -5.6779e-05, -7.2223e-05],
        [ 9.0776e-05,  3.8048e-04,  1.5636e-04,  ..., -2.6650e-04,
          1.0240e-04,  1.2402e-04],
        ...,
        [ 5.3351e-05, -2.1889e-05,  1.9218e-04,  ..., -1.5907e-04,
         -1.0281e-04,  2.1882e-04],
        [ 9.8477e-05,  1.7131e-04,  5.1956e-04,  ...,  4.5358e-04,
          4.0474e-04, -8.5452e-05],
        [ 3.1593e-04, -2.1254e-04,  1.6734e-05,  ..., -6.0096e-05,
          4.1705e-05, -2.0867e-04]])

In [110]:
(ov_weighted + qkvo_2ndorder - qkvo_jl_2ndorder)

tensor([[-1.2131e-03,  1.7963e-03, -6.9012e-04,  ..., -1.9283e-03,
         -2.1450e-03, -5.4877e-04],
        [ 1.5740e-03,  8.1124e-04,  2.8137e-04,  ...,  4.8877e-04,
          2.4495e-04,  8.5475e-04],
        [-1.2432e-03, -1.3063e-03, -2.8456e-03,  ..., -1.8305e-03,
         -2.1957e-03, -7.5811e-04],
        ...,
        [ 1.9592e-03,  3.9733e-04,  1.7152e-03,  ...,  4.2522e-04,
         -1.9404e-04, -1.6459e-03],
        [ 2.5624e-03, -1.5457e-03,  9.0923e-04,  ...,  2.2632e-03,
         -6.6722e-04, -2.1961e-03],
        [ 1.9412e-04, -2.3323e-03,  3.8895e-03,  ..., -2.7402e-03,
         -9.5719e-05,  1.0051e-03]])

In [111]:
# Compute explained variance between numeric Jacobian and OV circuit
x = numeric_jac.flatten()
y = (ov_weighted + qkvo_2ndorder - qkvo_jl_2ndorder).flatten()

# Calculate correlation coefficient
corr = torch.corrcoef(torch.stack([x, y]))[0, 1]
print(f"Correlation: {corr:.4f}")

Correlation: 0.8212


Calculate the residual from QK circuit 

In [62]:
Wqk.shape

torch.Size([12, 768, 768])

In [68]:
X_Hqk = einsum(X[0, :, :].detach(), Wqk, "T dq, H dq dk -> H T dk")
X_Hov = einsum(X[0, :, :].detach(), Wov, "T dv, H do dv -> H T do")

In [75]:
a_ila_ij = einsum(attn_weights[0, :, i, :], attn_weights[0, :, i, j], "H al, H -> H al")
a_ildelta_jl = einsum(attn_weights[0, :, i, :], torch.eye(seq_len)[j, :], "H al, al -> H al")
a_ila_ij.shape

torch.Size([12, 10])

In [79]:
(a_ildelta_jl - a_ila_ij).shape

torch.Size([12, 10])

In [86]:
QK_residual = einsum(a_ildelta_jl - a_ila_ij, X_Hov, X_Hqk[:, i, :], "H Tl, H Tl do, H dk -> do dk")
QK_residual.shape

torch.Size([768, 768])

In [87]:
numeric_jac

tensor([[-5.3543e-04,  2.8788e-03, -2.3455e-04,  ..., -4.0922e-03,
         -1.7396e-03,  2.4052e-03],
        [ 1.8423e-03,  1.1572e-03,  2.5181e-03,  ...,  2.8669e-03,
         -2.1119e-03,  3.6766e-04],
        [-3.3756e-04, -7.4580e-04, -3.8852e-03,  ..., -2.9074e-03,
         -2.1072e-03, -3.7693e-04],
        ...,
        [-5.5887e-04,  1.0784e-03,  2.4282e-03,  ...,  2.7294e-04,
         -2.1278e-03, -2.5460e-03],
        [ 1.3023e-03, -4.7290e-03, -1.5914e-03,  ...,  3.2138e-03,
         -9.9539e-05, -1.8348e-03],
        [-6.5471e-04, -2.0343e-03,  3.0956e-03,  ..., -2.5304e-03,
         -7.6103e-04, -1.2569e-03]])

In [89]:
ov_weighted

tensor([[-6.6245e-04,  3.0233e-03, -1.0986e-03,  ..., -3.4217e-03,
         -2.3682e-03, -2.9564e-04],
        [ 1.2183e-03,  3.7412e-04,  3.4833e-05,  ..., -4.2435e-04,
         -3.8739e-04,  9.8769e-04],
        [ 4.2588e-04,  8.1284e-04, -1.3083e-03,  ..., -2.4901e-03,
         -2.5802e-03, -1.4192e-03],
        ...,
        [ 1.3703e-03, -7.1919e-05,  1.0638e-03,  ...,  1.8979e-04,
         -8.5865e-05, -1.3210e-03],
        [ 1.6400e-03, -2.7432e-03,  1.2730e-03,  ...,  1.4649e-03,
         -1.7425e-03, -4.9444e-04],
        [ 1.3888e-03, -2.0905e-03,  3.2543e-03,  ..., -3.8761e-03,
         -1.3961e-03,  9.9900e-04]])

In [91]:
import math
QK_residual / math.sqrt(head_dim) + ov_weighted

tensor([[-1.5241e-03,  2.7582e-03, -1.8893e-03,  ..., -3.0721e-03,
         -3.1361e-03, -3.0713e-04],
        [ 8.0075e-04,  1.3873e-03,  8.6474e-04,  ..., -4.0745e-04,
         -4.8818e-04,  1.4165e-03],
        [ 7.1090e-04, -3.4896e-05, -1.8774e-03,  ..., -3.0005e-03,
         -3.0440e-03, -1.2251e-03],
        ...,
        [ 2.3441e-03, -4.1954e-04,  1.4652e-03,  ...,  4.7644e-04,
          8.0550e-04, -2.0839e-03],
        [ 2.4550e-03, -1.8972e-03,  1.7842e-03,  ...,  2.2527e-03,
         -7.0341e-04, -2.3144e-03],
        [ 1.7686e-04, -1.4415e-03,  3.3979e-03,  ..., -3.6622e-03,
         -1.7878e-03,  1.5301e-03]])

In [77]:
a_ildelta_jl

tensor([[0.0000, 0.0000, 0.1813, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0612, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.1224, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.1070, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.1468, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.1626, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.1022, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0469, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.1419, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0576, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0

In [ ]:
a_ildelta_jl[0]

In [72]:
a_ildelta_jl.shape

torch.Size([12, 10, 10])

#### QK circuit only

In [112]:
# Choose token indices and head to inspect
i = 0  # query position
j = 2  # key position

def token_mapping_QKonly(xj, ):
    """In this version, we only consider jacobian through the value projection"""
    X1 = X.clone().detach().requires_grad_(False)
    X2 = X.clone().detach().requires_grad_(False)
    X2[0, j, :] = xj
    attn_output, attn_weights = mha(X2, X2, X1, need_weights=True, average_attn_weights=False)
    return attn_output[0, i, :]

X = torch.randn(batch_size, seq_len, d_model, requires_grad=True)
with torch.no_grad():
    attn_output, attn_weights = mha(X, X, X, need_weights=True, average_attn_weights=False)

numeric_jac_qkonly = torch.autograd.functional.jacobian(token_mapping_QKonly, X[0, j])

In [123]:
# the following predicts the QK circuit contribution to the Jacobian
X_Hqk = einsum(X[0, :, :].detach(), Wqk, "T dq, H dq dk -> H T dk")
X_Hov = einsum(X[0, :, :].detach(), Wov, "T dv, H do dv -> H T do")
qkvo_2ndorder = einsum(attn_weights[0, :, i, j], X_Hqk[:, i, :], X_Hov[:, j, :], "H, H dk, H do -> do dk") / math.sqrt(head_dim)
qkvo_jl_2ndorder = einsum(attn_weights[0, :, i, j], attn_weights[0, :, i, :], X_Hov[:, :, :], X_Hqk[:, i, :], "H, H Tl, H Tl do, H dk -> do dk") / math.sqrt(head_dim)
assert torch.allclose(numeric_jac_qkonly, qkvo_2ndorder - qkvo_jl_2ndorder)

In [113]:
numeric_jac_qkonly

tensor([[-1.7278e-03, -6.9233e-04,  1.7862e-03,  ...,  2.6447e-04,
         -3.9981e-04, -9.2942e-05],
        [ 4.2892e-05, -2.8013e-04, -4.4854e-04,  ..., -5.0583e-04,
          5.6668e-04, -2.7199e-04],
        [-1.6954e-03, -1.7363e-04,  1.7467e-04,  ..., -1.1139e-04,
          5.2712e-05,  8.0204e-04],
        ...,
        [ 6.9698e-04,  1.1611e-03, -4.2240e-04,  ...,  3.6185e-04,
         -2.3370e-04, -6.7715e-04],
        [ 7.5311e-04,  9.8203e-04, -7.8470e-04,  ...,  5.1064e-04,
         -6.4541e-04, -6.6169e-04],
        [-4.2168e-04,  3.0571e-04, -1.7050e-03,  ..., -9.2496e-04,
          5.3787e-04,  1.1532e-03]])

In [114]:
X_Hqk = einsum(X[0, :, :].detach(), Wqk, "T dq, H dq dk -> H T dk")
X_Hov = einsum(X[0, :, :].detach(), Wov, "T dv, H do dv -> H T do")

2nd term

In [115]:
qkvo_2ndorder = einsum(attn_weights[0, :, i, j], X_Hqk[:, i, :], X_Hov[:, j, :], "H, H dk, H do -> do dk") / math.sqrt(head_dim)

Third term 

In [116]:
qkvo_jl_2ndorder = einsum(attn_weights[0, :, i, j], attn_weights[0, :, i, :], X_Hov[:, :, :], X_Hqk[:, i, :], "H, H Tl, H Tl do, H dk -> do dk") / math.sqrt(head_dim)
qkvo_jl_2ndorder.shape

torch.Size([768, 768])

In [118]:
- qkvo_jl_2ndorder + qkvo_2ndorder

tensor([[-1.7278e-03, -6.9233e-04,  1.7862e-03,  ...,  2.6447e-04,
         -3.9981e-04, -9.2943e-05],
        [ 4.2892e-05, -2.8013e-04, -4.4854e-04,  ..., -5.0583e-04,
          5.6668e-04, -2.7199e-04],
        [-1.6954e-03, -1.7363e-04,  1.7467e-04,  ..., -1.1139e-04,
          5.2712e-05,  8.0204e-04],
        ...,
        [ 6.9698e-04,  1.1611e-03, -4.2240e-04,  ...,  3.6185e-04,
         -2.3370e-04, -6.7715e-04],
        [ 7.5311e-04,  9.8203e-04, -7.8470e-04,  ...,  5.1064e-04,
         -6.4541e-04, -6.6169e-04],
        [-4.2168e-04,  3.0571e-04, -1.7050e-03,  ..., -9.2496e-04,
          5.3787e-04,  1.1532e-03]])

#### VO circuit only

In [55]:
# Choose token indices and head to inspect
i = 0  # query position
j = 2  # key position

def token_mapping_Vonly(xj, ):
    """In this version, we only consider jacobian through the value projection"""
    X1 = X.clone().detach().requires_grad_(False)
    X2 = X.clone().detach().requires_grad_(False)
    X2[0, j, :] = xj
    attn_output, attn_weights = mha(X1, X1, X2, need_weights=True, average_attn_weights=False)
    return attn_output[0, i, :]

X = torch.randn(batch_size, seq_len, d_model, requires_grad=True)
with torch.no_grad():
    attn_output, attn_weights = mha(X, X, X, need_weights=True, average_attn_weights=False)

numeric_jac_vonly = torch.autograd.functional.jacobian(token_mapping_Vonly, X[0, j])

# Note in this case the Jacobian is EXACTLY equal to the weighted sum of the OV matrix per head. 
ov_weighted = einsum(attn_weights[0, :, i, j], Wov, "H, H do dv -> do dv")
assert torch.allclose(numeric_jac_vonly, ov_weighted)

In [59]:
attn_weights[0, :, i, j]

tensor([0.1813, 0.0612, 0.1224, 0.1070, 0.1468, 0.1626, 0.1022, 0.0469, 0.1419,
        0.0576, 0.0637, 0.0361])

In [56]:
numeric_jac_vonly.shape

torch.Size([768, 768])

### Scratch

In [ ]:
# Choose token indices and head to inspect
i = 0  # query position
j = 2  # key position
h = 1  # head index
# Function to extract attention weight a_{ij} for head h
def wrapped_xj(xj):
    X2 = X.clone().detach().requires_grad_(True)
    X2 = X2 + X
    X2[j, 0] = xj
    _, w = mha(X2, X2, X2, need_weights=True, average_attn_weights=False)
    return w[0, h, i, j]

# Numeric Jacobian ∂a_{ij}/∂x_j via automatic differentiation
numeric_jac = torch.autograd.functional.jacobian(wrapped_xj, X[j, 0])

# Analytic Jacobian: a_{ij}(1 - a_{ij}) * (1/√head_dim) * W_k^hᵀ (W_q^h x_i)
with torch.no_grad():
    # Extract in-projection weights [W_q; W_k; W_v]
    Wqkv = mha.in_proj_weight  # shape (3*d_model, d_model)
    W_q = Wqkv[:d_model]
    W_k = Wqkv[d_model:2*d_model]
    # Head-specific slices
    Wq_h = W_q[h*head_dim:(h+1)*head_dim]
    Wk_h = W_k[h*head_dim:(h+1)*head_dim]
    
    # Compute projected query for position i
    qi = Wq_h @ X[i, 0]
    # Compute score derivative w.r.t. x_j
    de_dxj = (Wk_h.t() @ qi) / (head_dim ** 0.5)
    # Attention weight
    a_ij = attn_weights[0, h, i, j]
    # Since δ_{jj}=1
    analytic_jac = a_ij * (1 - a_ij) * de_dxj

# Print results
print("Numeric Jacobian ∂a_{ij}/∂x_j:\n", numeric_jac)
print("\nAnalytic Jacobian ∂a_{ij}/∂x_j:\n", analytic_jac)
print("\nDifference (numeric - analytic):\n", numeric_jac - analytic_jac)

Numeric Jacobian ∂a_{ij}/∂x_j:
 tensor([-2.0582e-02,  3.6188e-03,  2.8728e-04, -6.2182e-03, -3.0543e-03,
         4.3134e-03, -4.4104e-03,  1.1021e-02,  1.1855e-02,  4.1377e-03,
         8.4561e-03,  1.5518e-02, -1.0559e-02, -3.9157e-03, -5.3817e-03,
         2.2429e-02,  5.0443e-03,  8.9291e-03,  3.9322e-03, -2.0007e-02,
         3.9790e-03, -1.0476e-03, -3.4965e-03,  5.2969e-03,  1.8942e-02,
         1.2890e-02, -3.9444e-03, -1.5408e-02, -4.7668e-04,  3.2827e-03,
         1.0726e-03, -4.4947e-03, -2.7808e-03,  3.4535e-03,  7.8083e-03,
         1.9366e-03, -3.5040e-03,  8.4076e-05,  8.6500e-04,  1.0419e-02,
        -3.3230e-03, -9.0689e-04, -4.9903e-03, -2.0160e-03, -8.8646e-03,
        -1.5265e-02, -3.0279e-03, -5.1183e-03, -1.4756e-02,  8.2104e-03,
        -2.0399e-03,  1.4889e-02, -7.4006e-03, -1.1540e-02, -2.9421e-03,
        -2.3630e-03,  5.1002e-03, -2.9835e-03, -1.5015e-02,  3.5388e-03,
        -1.1657e-02, -5.7463e-03,  9.8745e-03, -8.9335e-04])

Analytic Jacobian ∂a_{ij}/∂x_j